### Required imports

In [ ]:
#!pip install opencv-python

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings("ignore")

In [1]:
import os 
path = '../'
print(path)
import sys 
sys.path.insert (0, path) 

../


In [2]:
import imageio as io
import cv2
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

### Read data

In [5]:
data = pd.read_csv('../airbnb-listings.csv', sep = ';')

In [5]:
data.shape

(14001, 89)

### Load images

In [ ]:
images_paths = [[i, img_url] for i, img_url in enumerate(data['Thumbnail Url'])]

# Tensor to save the images
loaded_images = np.zeros((len(images_paths), 224, 224, 3), dtype=np.uint8)

# Tensor to check which images were not saved
was_loaded = np.zeros(len(images_paths))

In [ ]:
# This function downloads the images and returns them with the index where to save them
def get_image(data_url, target_size=(224, 224)):
    idx, url = data_url
    try:
        img = io.imread(url)
        # Some images may be in black & white. I convert to to RGB as otherwise it generates error
        if img.ndim < 3:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        img = cv2.resize(img, dsize=target_size)
        return img, idx
    except IOError as err:
        return (None, idx)

In [ ]:
def main():
# Creating pool to load images. By default, it uses as much cores as your machine has
    with ProcessPoolExecutor() as executor:
        for (img, idx) in tqdm(executor.map(get_image, images_paths), total=len(images_paths)):
            # Loading the images
            try:
                if img is not None:
                    loaded_images[idx] = img
                    was_loaded[idx] = 1
                else:
                    was_loaded[idx] = 0
            except:
                    print("Problem with an instance!")

if __name__ == '__main__':
    main()
    
print('Finished!')
print(f'Number of images loaded: {sum(was_loaded)}/{len(images_paths)}')

In [ ]:
# Saving images. I'd recommend to save this to not repeat this process3
np.save('images.npy', loaded_images)
np.save('was_loaded.npy', was_loaded)